import pandas as pd

In [1]:
import pandas as pd

for split in range(5):
    path = f"facebase/data/Adience_256x256_resnet50_imagenet_noisy_dldl_v2/data_split{split}.csv"
    df = pd.read_csv(path, header=None)
    df[4] = df[3].astype(float)
    df[5] = 0.75

    df.to_csv(path, index=False, header=False)

In [ ]:
import pandas as pd
from collections import defaultdict

# List of CSV files
files = [f'facebase/data/Adience_256x256_resnet50_imagenet_noisy_dldl_v2_sync_1/data_split{i}.csv' for i in range(5)]

# Dictionary to store data based on id
data_dict = defaultdict(dict)

# Read files and store data
for file in files:
    df = pd.read_csv(file, header=None)  # No header in the files
    for _, row in df.iterrows():
        id = row[0]  # First column is id
        folder = row[2]  # Third column is folder
        mean = row[4]  # Fifth column is mean
        sigma = row[5]  # Sixth column is sigma
        
        # Only consider rows where folder is 0 or 1
        if folder in [0, 1]:
            if id in data_dict:
                # Check if mean or sigma differs for the same id
                if (data_dict[id]['mean'] != mean) or (data_dict[id]['sigma'] != sigma):
                    print(f"Difference found for id {id} in file {file}: mean or sigma does not match.")
            else:
                # Store mean and sigma for the id if it's not already in the dictionary
                data_dict[id] = {'mean': mean, 'sigma': sigma}

print("Validation complete.")

In [1]:
import torch

eval_file = "facebase/results/Adience_256x256_resnet50_imagenet_noisy_dldl_v2_sync_1_best_skip_new/split0/evaluation.pt"
ev = torch.load(eval_file)

ev.keys()

dict_keys(['config', 'split', 'error', 'log_history', 'posterior', 'true_label', 'predicted_label', 'id', 'folder'])

In [4]:
ev['log_history']

[{'trn_loss': 2.3320484046712964,
  'trn_error': 1.3863427639007568,
  'trn_loss_age': array(2.3320484),
  'trn_error_age': array(1.3863428, dtype=float32),
  'val_loss': 1.760980202568446,
  'val_error': 1.2926031351089478,
  'val_loss_age': array(1.7609802),
  'val_error_age': array(1.2926031, dtype=float32),
  'elapsed_minutes': 0.4947106083234151},
 {'trn_loss': 1.9904477628898636,
  'trn_error': 1.140770673751831,
  'trn_loss_age': array(1.99044776),
  'trn_error_age': array(1.1407707, dtype=float32),
  'val_loss': 1.7406978754023024,
  'val_error': 1.2486152648925781,
  'val_loss_age': array(1.74069788),
  'val_error_age': array(1.2486153, dtype=float32),
  'elapsed_minutes': 0.902175517876943},
 {'trn_loss': 1.8303885658355625,
  'trn_error': 1.0345988273620605,
  'trn_loss_age': array(1.83038857),
  'trn_error_age': array(1.0345988, dtype=float32),
  'val_loss': 1.675076447469131,
  'val_error': 1.2157055139541626,
  'val_loss_age': array(1.67507645),
  'val_error_age': array(1

In [6]:
a = torch.tensor([2587, 2384, 2333, 1936, 3852, 2185, 1223, 1202])
2587 / a.sum()

tensor(0.1461)

In [8]:
torch.log(torch.tensor([0.1461]))

tensor([-1.9235])

# Mean Prediction

In [ ]:
import torch
import os
import numpy as np

def get_mean_prediction(root):
    eval_files = [os.path.join(root, f"split{i}", "evaluation.pt") for i in range(5)]
    for i, eval_file in enumerate(eval_files):
        print(eval_file)
        ev = torch.load(eval_file)
        indices = {'trn': [j for j in range(len(ev['id'])) if ev['folder'][j] == 0],
                   'val': [j for j in range(len(ev['id'])) if ev['folder'][j] == 1],
                   'tst': [j for j in range(len(ev['id'])) if ev['folder'][j] == 2]}

        for label in sorted(list(set(ev['true_label']['age']))):
            for mode in ['trn', 'val', 'tst']:
                class_indices = [idx for idx in indices[mode] if ev['true_label']['age'][idx] == label]
                print(label, mode, np.mean(ev['predicted_label']['age'][class_indices]))

            print()

get_mean_prediction('facebase/results/Adience_256x256_resnet50_imagenet_dldl_v2_clean')

facebase/results/Adience_256x256_resnet50_imagenet_dldl_v2_clean/split0/evaluation.pt
0 trn 0.01723209477652127
0 val 0.10596026490066225
0 tst 0.17184265010351968

1 trn 1.0217717717717718
1 val 0.8025210084033614
1 tst 1.2

2 trn 2.01786941580756
2 val 1.9235412474849094
2 tst 2.75

3 trn 3.0939809926082367
3 val 3.5106837606837606
3 tst 3.607929515418502

4 trn 3.985975212002609
4 val 3.9701030927835053
4 tst 3.9924457034938623

5 trn 4.868441064638783
5 val 4.472275334608031
5 tst 4.48605577689243

6 trn 5.661855670103093
6 val 5.115384615384615
6 tst 4.846473029045643

7 trn 6.853118712273642
7 val 5.4491525423728815
7 tst 5.603112840466926

facebase/results/Adience_256x256_resnet50_imagenet_dldl_v2_clean/split1/evaluation.pt
0 trn 0.012442698100851343
0 val 0.2902829028290283
0 tst 0.11258278145695365

1 trn 1.0058290155440415
1 val 1.0670391061452513
1 tst 0.6890756302521008

2 trn 2.0257771038665657
2 val 2.203781512605042
2 tst 2.1046277665995974

3 trn 3.081858407079646
3 val

In [9]:
get_mean_prediction('facebase/results/Adience_256x256_resnet50_imagenet_noisy_dldl_v2_clean_carl')

facebase/results/Adience_256x256_resnet50_imagenet_noisy_dldl_v2_clean_carl/split0/evaluation.pt
0 trn 0.06560088202866593
0 val 0.0
0 tst 0.14285714285714285

1 trn 1.0458991723100075
1 val 0.9935691318327974
1 tst 1.2228070175438597

2 trn 2.071219512195122
2 val 1.9888059701492538
2 tst 2.8411764705882354

3 trn 3.1921296296296298
3 val 3.0048899755501224
3 tst 3.616740088105727

4 trn 4.019623875715454
4 val 3.9977238239757207
4 tst 4.048158640226629

5 trn 5.03580146460537
5 val 4.948275862068965
5 tst 4.581673306772909

6 trn 6.308035714285714
6 val 5.859504132231405
6 tst 4.842323651452282

7 trn 6.993055555555555
7 val 7.0
7 tst 5.879377431906615

facebase/results/Adience_256x256_resnet50_imagenet_noisy_dldl_v2_clean_carl/split1/evaluation.pt
0 trn 0.05085865257595773
0 val 0.0
0 tst 0.09271523178807947

1 trn 0.8691449814126394
1 val 1.0
1 tst 0.47058823529411764

2 trn 1.8860369609856262
2 val 1.9735973597359735
2 tst 1.8028169014084507

3 trn 3.0262172284644193
3 val 3.02215